<a href="https://colab.research.google.com/github/Tom-Jung/Tom-Jung/blob/main/Eurostat_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import os
import gzip
import urllib.request
def download_eurostat_dataset(dataset):
    url = "https://ec.europa.eu/eurostat/" + \
          "estat-navtree-portlet-prod/" + \
          "BulkDownloadListing?file=data/" + \
          dataset + ".tsv.gz"
    out_file = os.path.join(dataset + ".tsv")
    if os.path.exists(out_file):
        return
    try:
        with urllib.request.urlopen(url) as resp:
            with gzip.GzipFile(fileobj=resp) as data:
                file_content = data.read()
# Write to file in binary mode 'wb'
        with open(out_file, 'wb') as f:
            f.write(file_content)
    except Exception as e:
        print(e)
download_eurostat_dataset("teilm020")

In [41]:
def get_eurostat_dataset(dataset):
    download_eurostat_dataset(dataset)
    df = pd.read_csv(os.path.join(dataset + ".tsv"),
                     sep=",|\t| [^ ]?\t", na_values=":",
                     engine="python")
    df.columns = [x.split('\\')[0].strip(' ') for x in df.columns]
    return df
df = get_eurostat_dataset("teilm020")
df

,s_adj,age,sex,unit,geo,2021M10,2021M11,2021M12,2022M01,2022M02,2022M03,2022M04,2022M05,2022M06,2022M07,2022M08,2022M09
0,SA,TOTAL,F,PC_ACT,AT,5.9,5.2,4.7,4.4,4.8,4.2,4.3,4.4,4.1,4.2,4.9,5.1
1,SA,TOTAL,F,PC_ACT,BE,4.9,4.7,4.8,5.0,5.2,5.3,5.4,5.5,5.5,5.6,5.6,5.6
2,SA,TOTAL,F,PC_ACT,BG,4.2,4.2,4.3,4.5,4.6,4.5,4.2,4.2,4.2,4.2,4.3,4.3
3,SA,TOTAL,F,PC_ACT,CH,4.7,4.6,4.6,4.5,4.5,4.5,4.7,4.7,4.6,NaN,NaN,NaN
4,SA,TOTAL,F,PC_ACT,CY,7.3,7.8,7.7,7.3,7.2,7.2,7.4,7.6,8.5,9.5,10.0,9.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,SA,TOTAL,T,PC_ACT,SE,8.5,8.2,8.0,8.0,7.4,7.6,7.7,7.7,7.6,7.0,6.9,7.0
101,SA,TOTAL,T,PC_ACT,SI,4.7,4.7,4.4,4.1,4.0,4.1,4.4,4.5,4.4,4.2,4.1,4.1
102,SA,TOTAL,T,PC_ACT,SK,6.6,6.5,6.5,6.5,6.4,6.3,6.3,6.2,6.1,6.0,5.9,5.9
103,SA,TOTAL,T,PC_ACT,TR,11.2,11.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
def download_eurostat_dict(dictionary):
    dictionary = dictionary.lower()
    url = "https://ec.europa.eu/eurostat/" + \
          "estat-navtree-portlet-prod/BulkDownloadListing" + \
          "?sort=1&downfile=dic%2Fen%2F" +\
          dictionary + ".dic"
    out_file = os.path.join("cache", dictionary + ".dic")
    if os.path.exists(out_file):
        return
    try:
        with urllib.request.urlopen(url) as resp:
                file_content = resp.read().decode('utf-8')
        with open(out_file, 'w') as f:
            f.write(file_content)
    except Exception as e:
        print(e)
    
def get_eurostat_dictionary(dictionary, inverse=False):
    download_eurostat_dict(dictionary)
    filename = os.path.join("cache", dictionary + ".dic")
    try:
        with open(filename) as f:
            d = {}
            for line in f:
                if len(line) > 1:
                    row = line.split('\t')
                    d[row[0]] = row[1].strip()
        if inverse:
            d = {v: k for k, v in d.items()}
        return d
    except:
        return {}

In [53]:
def get_eurostat_dataset(dataset):
    download_eurostat_dataset(dataset)
    df = pd.read_csv(os.path.join("cache", dataset + ".tsv"),
                     sep=",|\t| [^ ]?\t", na_values=":", 
                     engine="python")
    df.columns = [x.split('\\')[0].strip(' ') for x in df.columns]
    # Now get the dictionary columns (first line, comma sep)
    with open(os.path.join("cache", dataset + ".tsv")) as f:
        first_line = f.readline()
    codes = first_line.split('\t')[0].split('\\')[0].split(',')
    # Replace codes with value
    for c in codes:
        code_list = get_eurostat_dictionary(c)
        df[c] = df[c].replace(code_list)
    df = df.set_index(codes).transpose()
    return df

In [54]:
df

,s_adj,age,sex,unit,geo,2021M10,2021M11,2021M12,2022M01,2022M02,2022M03,2022M04,2022M05,2022M06,2022M07,2022M08,2022M09
0,SA,TOTAL,F,PC_ACT,AT,5.9,5.2,4.7,4.4,4.8,4.2,4.3,4.4,4.1,4.2,4.9,5.1
1,SA,TOTAL,F,PC_ACT,BE,4.9,4.7,4.8,5.0,5.2,5.3,5.4,5.5,5.5,5.6,5.6,5.6
2,SA,TOTAL,F,PC_ACT,BG,4.2,4.2,4.3,4.5,4.6,4.5,4.2,4.2,4.2,4.2,4.3,4.3
3,SA,TOTAL,F,PC_ACT,CH,4.7,4.6,4.6,4.5,4.5,4.5,4.7,4.7,4.6,NaN,NaN,NaN
4,SA,TOTAL,F,PC_ACT,CY,7.3,7.8,7.7,7.3,7.2,7.2,7.4,7.6,8.5,9.5,10.0,9.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,SA,TOTAL,T,PC_ACT,SE,8.5,8.2,8.0,8.0,7.4,7.6,7.7,7.7,7.6,7.0,6.9,7.0
101,SA,TOTAL,T,PC_ACT,SI,4.7,4.7,4.4,4.1,4.0,4.1,4.4,4.5,4.4,4.2,4.1,4.1
102,SA,TOTAL,T,PC_ACT,SK,6.6,6.5,6.5,6.5,6.4,6.3,6.3,6.2,6.1,6.0,5.9,5.9
103,SA,TOTAL,T,PC_ACT,TR,11.2,11.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
 # Get names of all levels
print(df.columns.names)
['s_adj', 'age', 'sex', 'unit', 'geo']
# Obtaining all unique values for level 2 ('sex')
print(df.columns.get_level_values(0).unique())
# Index(['Females', 'Males', 'Total'], dtype='object', name='sex')
#Selecting unemployment data for all sexes combined
df.loc[:, ('Seasonally adjusted data, ...',
               'Total', 
               'Total',
               'Percentage of population in the labour force')]

[None]
Index(['s_adj', 'age', 'sex', 'unit', 'geo', '2021M10', '2021M11', '2021M12',
       '2022M01', '2022M02', '2022M03', '2022M04', '2022M05', '2022M06',
       '2022M07', '2022M08', '2022M09'],
      dtype='object')


KeyError: ignored